In [1]:
# Import pandas and numpy
import pandas as pd
import numpy as np
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

In [2]:
# Load the dataset into a DataFrame
sms_text_df = pd.read_csv('Resources/SMSSpamCollection.csv')
sms_text_df.head()

,label,text_message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [3]:
# Check for missing values. 
sms_text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   label         5572 non-null   object
 1   text_message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
#  Get the number of "ham" and "spam" from the "label" column:
sms_text_df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

## Split the data into train & test sets:

In [5]:
# Set the features variable to the text message. 
X = sms_text_df['text_message']  
# Set the target variable to the "label" column.
y = sms_text_df['label']

# Split data into training and testing and set the test_size = 33%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
# Create an instance of the TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Transform the data and use the original X_train set.
X_train_tfidf = vectorizer.fit_transform(X_train) 
X_train_tfidf.shape

(3733, 6823)

In [7]:
# What stopwords are in the scikit-learn's built-in list
from sklearn.feature_extraction import text
print(text.ENGLISH_STOP_WORDS)

frozenset({'over', 'less', 'further', 'among', 'through', 'under', 'either', 'nine', 'yourself', 'hereafter', 'latter', 'such', 'there', 'bottom', 'hereby', 'throughout', 'somehow', 'always', 'yourselves', 'full', 'our', 'him', 'hereupon', 'when', 'whole', 'done', 'nowhere', 'un', 'themselves', 'hundred', 'herein', 'con', 'whether', 'not', 'seeming', 'afterwards', 'whenever', 'thence', 'here', 'whither', 'do', 'becoming', 'whence', 'hers', 'as', 'indeed', 'my', 'after', 'mill', 'thus', 'were', 'anywhere', 'same', 'up', 'we', 'sincere', 'being', 'those', 'serious', 'front', 'herself', 'moreover', 'i', 'namely', 'due', 'can', 'eg', 'whoever', 'noone', 'with', 'nobody', 'perhaps', 'an', 'but', 'fire', 'by', 'former', 'beforehand', 'except', 'against', 'at', 'whereupon', 'wherein', 'cannot', 're', 'whom', 'nor', 'please', 'who', 'until', 'anyhow', 'have', 'few', 'himself', 'might', 'twelve', 'no', 'along', 'although', 'than', 'his', 'may', 'something', 'around', 'somewhere', 'eight', 'toge

In [8]:
# Create a list to hold the words using the vectorizer.get_feature_names_out()
words = list(vectorizer.get_feature_names_out())
# Create a list to hold the frequency using np.ravel(X.sum(axis=0))
frequency = list(np.ravel(X_train_tfidf.sum(axis=0)))

In [9]:
# Create a DataFrame of the TF–IDF weights for each word in the working corpus.
messages_df = pd.DataFrame({
    "Word": words,
    "Frequency": frequency})

# Sort the DataFrame by word frequency in descending order and reset the index.
messages_df = messages_df.sort_values(by=["Frequency"], ascending=False).reset_index(drop=True)

# Display the DataFrame
messages_df.head(10)

,Word,Frequency
0,ok,69.321087
1,ll,54.283266
2,just,50.524804
3,ur,44.971124
4,come,44.652090
5,gt,44.163010
6,lt,44.052110
7,good,42.019014
8,time,39.633440
9,know,38.782320


In [10]:
# Display the DataFrame
messages_df.tail(10)

,Word,Frequency
6813,sympathetic,0.074647
6814,healer,0.074647
6815,dependable,0.074647
6816,determined,0.074647
6817,venaam,0.074647
6818,driver,0.074647
6819,aaniye,0.074647
6820,stylist,0.074647
6821,exterminator,0.074647
6822,pest,0.074647


In [11]:
# Train the data on LinearSVC classifier.
linear_svc_model = LinearSVC()
# Fit the model to the transformed  data,
linear_svc_model.fit(X_train_tfidf,y_train)

LinearSVC()

In [12]:
# Determine predictions. 
# Run this to show that we can't run predcitions on our linear SVC model. 
# We have to transform our testing data the same way as we transformed the training data. 
predictions = linear_svc_model.predict(X_test)

ValueError: could not convert string to float: 'Squeeeeeze!! This is christmas hug.. If u lik my frndshp den hug me back.. If u get 3 u r cute:) 6 u r luvd:* 9 u r so lucky;) None? People hate u:'

In [13]:
# Transform the testing data like we did with the training data.
X_test_tfidf = vectorizer.transform(X_test) 
# Make predictions 
predictions = linear_svc_model.predict(X_test_tfidf)
print(predictions[:30])

['ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'ham']


In [14]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % linear_svc_model.score(X_train_tfidf, y_train))
print('Test Accuracy: %.3f' % linear_svc_model.score(X_test_tfidf, y_test))

Train Accuracy: 1.000
Test Accuracy: 0.989


## For efficiency,  build a Pipeline with the vectorizer and SVM model. 

In [15]:
# Build a pipeline to transform the test set to compare to the training set. 
text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                     ('clf', LinearSVC()),
])

# Fit the model to the transformed data.
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('clf', LinearSVC())])

In [16]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % text_clf.score(X_train, y_train))
print('Test Accuracy: %.3f' % text_clf.score(X_test, y_test))

Train Accuracy: 1.000
Test Accuracy: 0.989


## Test the classifier and display results

In [17]:
# Form a prediction set
message_predictions = text_clf.predict(X_test)
print(message_predictions[0:30])

['ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'ham']


In [18]:
# Create the confusion matrix on the test data and predictions
print(metrics.confusion_matrix(y_test,message_predictions))

# Print a classification report
print(metrics.classification_report(y_test,message_predictions))

# Print the overall accuracy
print(metrics.accuracy_score(y_test,message_predictions))

[[1587    6]
 [  15  231]]
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.94      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.97      1839
weighted avg       0.99      0.99      0.99      1839

0.9885807504078303


Using the text of the messages, our model performed exceedingly well; it correctly predicted spam **98.9%** of the time!<br>
Now let's apply what we've learned to a text classification project involving positive and negative movie reviews.

In [19]:
# Create some random text messages. 
text_1 = """You are a lucky winner of $5000!!"""
text_2 = """You won 2 free tickets to the Super Bowl."""
text_3 = """You won 2 free tickets to the Super Bowl text us to claim your prize"""
text_4 = """Thanks for registering. Text 4343 to receive free updates on medicare"""

In [20]:
# Send the text messages to transform the data and predict the classification.
print(text_clf.predict([text_1]))
print(text_clf.predict([text_2]))
print(text_clf.predict([text_3]))
print(text_clf.predict([text_4]))

['ham']
['ham']
['spam']
['spam']
